In [ ]:
from getdata import x_train, x_test, y_train, y_test, x_train_all, x_test_all

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# Previous step suggested most models were comparible but linear SVC 
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

In [ ]:
# TODO: Add dimensionality reduction to this pipeline?  
# http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html
# http://scikit-learn.org/stable/auto_examples/text/document_clustering.html
# https://medium.com/@adi_enasoaie/easy-lsi-pipeline-using-scikit-learn-a073f2484408

clf = Pipeline([
    ('vect', CountVectorizer(ngram_range = (1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(C=0.25, kernel="linear"))
])

In [ ]:
clf = clf.fit(x_train, y_train)

In [ ]:
from sklearn import metrics
y_pred = clf.predict(x_test)
print(metrics.classification_report(y_test, y_pred, digits=4))
metrics.classification_report

In [ ]:
metrics.confusion_matrix(y_test, y_pred, labels=[0, 1])

In [ ]:
import pandas as pd
df = pd.DataFrame({"y_test":y_test, "y_pred": y_pred, "x" : x_test, "x_c" : x_test_all["category_concat_cat"]})

f1 = df["y_test"] == 0
f2 = df["y_pred"] == 1

df[f1 & f2]

In [ ]:

f1 = df["y_test"] == 1
f2 = df["y_pred"] == 0
df[f1 & f2].sample(10)

In [ ]:
sum(y_train)